In [19]:
import numpy as np 
import matplotlib.pyplot as plt
import sklearn as sk
import skimage as ski
import tensorflow as tf

## Data Handling

In [20]:
cifar = tf.keras.datasets.cifar10.load_data()
(x_train, y_train), (x_test, y_test) = cifar

In [47]:
# Taking a number of data points from each class
def get_N_from_each_class(N, percentage_of_testing_data, x_train, y_train, x_test, y_test):
    # TODO
    
    selected_indices = []
    for class_label in range(10):
        class_indices = [i for i in range(len(y_train)) if y_train[i][0] == class_label]
        selected_indices.append(class_indices[:N])
        
    x_train = x_train[selected_indices]
    y_train = y_train[selected_indices]
    x_train = x_train.reshape(-1, 32, 32, 3)
    y_train = y_train.reshape(-1, 32, 32, 3)
    
    
    selected_indices = []
    for class_label in range(10):
        class_indices = [i for i in range(len(y_test)) if y_test[i][0] == class_label]
        selected_indices.append(class_indices[:N*percentage_of_testing_data])
        
    x_test = x_test[selected_indices]
    y_test = y_test[selected_indices]
    x_test = x_test.reshape(-1, 32, 32, 3)
    y_test = y_test.reshape(-1, 32, 32, 3)
    
    
    # 
    # new_x_train = np.empty(x_train.shape)
    # new_y_train = None
    # 
    # new_x_test = None
    # new_y_test = None
    # 
    # for class_index in range(10):
    #     # Getting all data from that class from x_train
    #     # class_x_train = [i for i in x_train if y_train[i] == class_index]
    #     class_x_train = np.where()
    #     
    #     # Picking only N random datapoint from them
    #     class_x_train = class_x_train[:N]
    #     
    #     class_y_train = y_train[y_train.flatten() == class_index]
    #     class_y_train = class_y_train[:N]
    #     
    #     # Adding the data to the new data
    #     new_x_train += class_x_train
    #     new_y_train += class_y_train
    #     
    # # Doing the same thing with testing data as the previous for loop
    # for class_index in range(10):
    #     class_x_test = [i for i in x_test if y_test[i] == class_index]
    #     class_x_test = class_x_test[np.random.choice(len(class_x_test), size=N*percentage_of_testing_data, replace=False)]
    #     
    #     class_y_test = [i for i in y_test if y_test[i] == class_index]
    #     class_y_test = class_y_test[np.random.choice(len(class_y_test), size=N*percentage_of_testing_data, replace=False)]
    #     
    #     new_x_test += class_x_test
    #     new_y_test += class_y_test
    # 
    # # Replacing original data with new data
    # x_train = new_x_train
    # y_train = new_y_train
    # x_test = new_x_test
    # y_test = new_y_test

n_samples_from_each_class = 100
get_N_from_each_class(n_samples_from_each_class, 0.7, x_train, y_train, x_test, y_test)
print(len(x_train))
print(x_train)

# print(y_train)


ValueError: cannot reshape array of size 1000 into shape (32,32,3)

In [23]:
# extrating the features from the images
# method used: Central Moments

def extract_image_features(image , C = 3):
    '''
    Extracts the features from the image using the central moments method of order C
    It turns the image to grayscale, then calculates the central moments of order C
    
    Parameters:
    image: the image to be processed
    C: the moments order

    Returns:
    features: a vector of the features extracted from the image
    '''

    # turning the image to grayscale
    image = ski.color.rgb2gray(image)

    # calculating the central moments
    cal_moments = ski.measure.moments_central(image, order = C)

    # create an np array that contains zeroes 
    features = np.zeros(shape = int((C + 1) * ( C + 2) / 2))


    # turn the 2D array to 1D by putting only the needed values based on this equation 
    # p + q <= C 
    # if C = 3 the needed numbers are : 00 01 10 11 20 02 21 12 03 30 , size = 10 s
    current = 0;
    for i in range (C + 1):
        for j in range (C + 1):
            if(i + j <= C):
                features[current] = cal_moments[i][j]
                current = current + 1

                
    return features    



In [24]:
#Display a sample of images
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    ax.imshow(x_train[i])
    ax.axis('off')
    ax.set_title(f'Label: {y_train[i][0]}')
    
plt.tight_layout()
plt.show()


##### SVM 
<b>(An Example for Multi-Class Classification using SVM)</b>

https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html

- Here is what we going to do for the SVM:
    * We will use the SVM implementation in sklearn
    * The parameters of the SVM will be chosen using grid search with cross validation
        * Degree for the polynomial kernel
        * C parameter for the SVM
            * C (Regularization Parameter): The `C` parameter is a positive scalar that controls the regularization strength. It influences the optimization process by balancing the desire to fit the training data well against the goal of keeping the model simple and avoiding overfitting.
            * Smaller values of C result in a wider margin, allowing more training points to be misclassified.
            * Larger values of C result in a narrower margin, forcing the optimizer to classify all training points correctly.
        * Gamma parameter for `rbf` , `poly` and `sigmoid` kernels
        * we will use both OVR and OVO strategies (`decision_function_shape` parameter)
    
    * We will Grid Search to find the best parameters for the SVM
    * We will use the `make_pipeline` 
    * We will use the `StandardScaler` to scale the data 

##### KNN 

( An Example for KNN Classification )

https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html#sphx-glr-auto-examples-neighbors-plot-classification-py

* The parameter we are going to experiment with the value of K (number of neighbors)
* The same thing as in SVM we are going to use `make_pipeline` and `StandardScaler` 